The VNN forum dataset has been:
* cleand from url
* chunked into max token lengt 256 using tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
* embeddings were calculated using model = SentenceTransformer('all-MiniLM-L6-v2')
* labeled with zeroshot classification (classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli"))

Text chunks with top scores from zeroshot classification were assesed manually till 100 positive and 100 negative labels were identified. \

Hypothesis for labeling is "this text is racist". \
*Racism is based on the idea that there are different human races. For the current example only hatered white people vs people of colour is considered (thus exluding for example antisemitism)*

In the following notebook df is divided into df_labeled and df_unlabeled (remaining)

In [4]:
from pathlib import Path
import pandas as pd

In [ ]:
# Set display options to make the text wrap in dataframes and display all rows
# pd.set_option('display.max_colwidth', None)  # Set to None to allow text to wrap
# pd.set_option('display.expand_frame_repr', False)  # Prevent the DataFrame from expanding to the available width
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

In [2]:
__file__ = '/home/dorota/projects/python/investigations/dorota_lia/active_learning/active_learning.ipynb' #TODO remove when working from .py as __file__ is defined in .py but not .ipunb

In [5]:
df = pd.read_csv(Path(__file__).parent / 'data/df_10k_chunked_zeroshot_racism.tsv',  sep = '\t')
df.shape

(12515, 15)

In [ ]:
# list of manually annotated indexes with 1 (racist) and 0 (not racist)
idx_1 = [3, 5, 8, 9, 11, 13, 15, 17, 18, 19, 22, 24, 25, 26, 27, 28, 31, 32, 36, 37, 41, 43, 44, 46, 47, 48, 50, 51, 
52, 53, 54, 56, 57, 60, 61, 62, 65, 66, 68, 72, 73, 76, 79, 80, 81, 83, 84, 85, 88, 93, 97, 103, 105, 106, 109, 111, 114, 115, 116, 117, 118, 119, 120, 123, 124, 126, 
127, 130, 131, 133, 134, 135, 140, 141, 142, 143, 144, 148, 149, 152, 153, 154, 155, 156, 164, 165, 166, 174, 176, 182, 183, 184, 186, 187, 189, 190, 191, 192, 203, 214]
idx_0 = [4, 12, 14, 16, 20, 30, 35, 49, 55, 63, 64, 78, 91, 95, 96, 99, 112, 113, 138, 150, 169, 172, 175, 291, 299, 2007, 2018, 2038, 2043, 2044, 2047, 2049, 2052, 2061, 2063, 2067, 2077, 2078, 2084, 2089, 
2091, 2093, 2099, 2101, 2105, 2112, 2114, 2117, 2126, 2127, 2130, 2804, 2813, 2815, 2816, 2829, 2834, 2859, 2879, 2882, 2886, 2891, 2907, 2945, 2949, 
2957, 2960, 2967, 2980, 2981, 2988, 2990, 2993, 2996, 3501, 3523, 3530, 3532, 3534, 3540, 3542, 3562, 3563, 3565, 3566, 3568, 3569, 3574, 5000, 5001, 5002, 5003, 9000, 9001, 9007, 9009, 9012, 9014, 9017, 9191 ]

In [ ]:
# combine indexes to one sorted list
idx_labeled = list(idx_1 + idx_0)
idx_labeled.sort()

In [ ]:
# create dataframe with labeled rows...
df_labeled = df.loc[idx_labeled]
df_labeled.loc[idx_1, 'racist_text'] = 1
df_labeled.loc[idx_0, 'racist_text'] = 0
df_labeled['racist_text'] = df_labeled['racist_text'].astype(int)
print(df_labeled.shape)
df_labeled.iloc[:, [0, 10, 11]  + list(range(13, len(df_labeled.columns)))].head()

In [ ]:
# ... and a dataframe with ramaining unlabeled rows
df_unlabeled = df[~df.index.isin(df_labeled.index)]
print(df_unlabeled.shape)
df_unlabeled.iloc[:, [0, 10, 11]  + list(range(13, len(df_unlabeled.columns)))].head()

In [ ]:
df_labeled.to_csv(Path(__file__).parent / 'data/df_labeled_racism.tsv', sep='\t', index=True)
df_unlabeled.to_csv(Path(__file__).parent / 'data/df_unlabeled_racism.tsv', sep='\t', index=True)